# StyleGAN2-ADA: training a model from scratch

## Prepare data

In [1]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [2]:
zip_location = '/content/drive/MyDrive/Work/Regular/Unetiq/ARNO/datasets/Bonn/filtered_128_50k.zip'
tf_data_location = '/content/drive/MyDrive/Work/Regular/Unetiq/ARNO/datasets/Bonn/tf_data/tf-dataset-bonn-labeled.zip'

### Generate

In [3]:
# %%capture

# # Unzip from Google Drive

# %cd /content
# !cp {zip_location} /content/
# !unzip /content/filtered_128_50k.zip

In [4]:
# # Process

# %cd /content
# !python stylegan2-ada/dataset_tool.py create_from_images /content/datasets/bonn /content/filtered_128/

In [5]:
# # Upload generated results to GDrive

# %cd /content/datasets
# !zip -r tf-dataset-bonn-labeled.zip bonn
# !cp /content/datasets/tf-dataset-bonn-labeled.zip {tf_data_location}

### Load

In [6]:
# Load from GDrive directly

!mkdir /content/datasets
!cp {tf_data_location} /content/datasets/
%cd /content/datasets
!unzip /content/datasets/tf-dataset-bonn-labeled.zip

Archive:  /content/datasets/tf-dataset-bonn-labeled.zip
   creating: bonn/
  inflating: bonn/bonn-r05.tfrecords  
  inflating: bonn/bonn-r04.tfrecords  
  inflating: bonn/bonn-r07.tfrecords  
  inflating: bonn/bonn-r06.tfrecords  
  inflating: bonn/bonn-rxx.labels    
   creating: bonn/.ipynb_checkpoints/
  inflating: bonn/bonn-r03.tfrecords  
  inflating: bonn/bonn-r02.tfrecords  


## Train new networks

In [1]:
# Configure ClearML

import os

os.environ["CLEARML_PROJECT_NAME"] = "Unetiq-ARNO"
os.environ["CLEARML_TASK_NAME"] = "StyleGAN2-ada-128x-v0"

In [8]:
# Set ClearML parameters

from clearml import Task, Logger

task = Task.init(project_name=os.environ["CLEARML_PROJECT_NAME"], task_name=os.environ["CLEARML_TASK_NAME"], continue_last_task=True)
task.connect_configuration('/root/clearml.conf')
logger = task.get_logger()
logger.set_default_upload_destination(uri='gs://clearml-bucket-0')

parameters = {
    'model': 'stylegan2-ada',
    'type':  'uncond',
    'data':  'bonn-128-50k'
}

parameters = task.connect(parameters)

task.mark_stopped()

ClearML Task: created new task id=054825127b064dd6b770fd3864d1e6a3
2021-01-25 23:23:32,769 - clearml.Repository Detection - WARNING - Can't get branch information for git repo in /usr/local/lib/python3.6/dist-packages
2021-01-25 23:23:32,779 - clearml.Repository Detection - WARNING - Can't get commit information for git repo in /usr/local/lib/python3.6/dist-packages
2021-01-25 23:23:32,789 - clearml.Repository Detection - WARNING - Can't get root information for git repo in /usr/local/lib/python3.6/dist-packages
2021-01-25 23:23:32,800 - clearml.Repository Detection - WARNING - Can't get status information for git repo in /usr/local/lib/python3.6/dist-packages
2021-01-25 23:23:32,818 - clearml.Repository Detection - WARNING - Can't get diff information for git repo in /usr/local/lib/python3.6/dist-packages
2021-01-25 23:23:32,830 - clearml.Repository Detection - WARNING - Can't get modified information for git repo in /usr/local/lib/python3.6/dist-packages
ClearML results page: https:/

In [2]:
# Switch to Tensorflow 1.x
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
%cd /content
!python stylegan2-ada/train.py \
 --outdir='/content/saves' \
 --snap=30 \
 --im_snap=10 \
 --gpus=1 \
 --data='/content/datasets/bonn' \
 --mirror=true \
 --metrics=none \
 --kimg=50000 \
 --cfg=auto_tuned

/content
ClearML Task: continuing previous task id=054825127b064dd6b770fd3864d1e6a3 Notice this run will not be reproducible!
ClearML results page: https://app.community.clear.ml/projects/945bd0c36255401197336ac56f0c2cc3/experiments/054825127b064dd6b770fd3864d1e6a3/output/log
tcmalloc: large alloc 4294967296 bytes == 0x7ab8000 @  0x7fb61ab00001 0x7fb617d434ff 0x7fb617d93b08 0x7fb617d97ac7 0x7fb617e361a3 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7fb429420000 @  0x7fb61aafe1e7 0x7fb617d4341e 0x7fb617d93c2b 0x7fb617d9430f 0x7fb617e360a3 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96

#### Resume from latest snapshot

Then,  **automatically** resume from the latest pickle snapshot.

Caveat: you will have to **manually** specify the augmentation strength, as found in `log.txt`.

In [14]:
!python stylegan2-ada/train.py \
 --outdir='/content/drive/My Drive/training-runs' \
 --snap=6 \
 --data='/content/datasets/steam' \
 --mirror=true \
 --metrics=none \
 --cfg=auto_no_ramp \
 --cfg_map=8 \
 --kimg=5000 \
 --resume=latest \
 --p=0.713 \
 --freezed=10 \
 

tcmalloc: large alloc 4294967296 bytes == 0x8250000 @  0x7f9e8d0c3001 0x7f9e8a2ec765 0x7f9e8a350bb0 0x7f9e8a352a4f 0x7f9e8a3e9048 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f9cbac20000 @  0x7f9e8d0c11e7 0x7f9e8a2ec5e1 0x7f9e8a350c78 0x7f9e8a350f37 0x7f9e8a3e8f28 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4
tcmalloc: large alloc 4294967296 bytes == 0x7f9bb9c1e000 @  0x7f9e8d0c11e7 0x7f9e8a2ec5e1 0x7f9e8a350c78 0x7f9e8a350f37 0x7f9e483560c5 0x7f9e47cd9902 0x7f9e47cd9eb2 0x7f9e47c92c3e 0x50a12f 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x588c8b 0x59fd0e 0x50d256 0x507b

# Generate images

In [8]:
%cd /content

/content


In [36]:
num_images = 20
network_pkl = '/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl'
out = 'gen/'

In [14]:
!python stylegan2-ada/generate.py --network={network_pkl} --outdir=out/notrunc0 --trunc=1.0 --seeds=0-{num_images-1}

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 0 (0/5) ...
Generating image for seed 1 (1/5) ...
Generating image for seed 2 (2/5) ...
Generating image for seed 3 (3/5) ...
Generating image for seed 4 (4/5) ...


In [15]:
!python stylegan2-ada/generate.py --network={network_pkl} --outdir=out/trunc3 --trunc=0.5 --seeds=0-{num_images-1}

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 0 (0/5) ...
Generating image for seed 1 (1/5) ...
Generating image for seed 2 (2/5) ...
Generating image for seed 3 (3/5) ...
Generating image for seed 4 (4/5) ...


In [37]:
!python stylegan2-ada/generate_video.py --network={network_pkl} --outdir=out --trunc=0.5 --seeds=0-{num_images-1} --interpolation_size=60

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.


In [39]:
!python stylegan2-ada/generate_video.py --network={network_pkl} --outdir=out/vid_notrunc1 --trunc=1 --seeds=0-{num_images-1} --interpolation_size=100

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.


In [19]:
!python stylegan2-ada/style_mixing.py --network={network_pkl} --outdir=out/mix0 --trunc=1.0 --rows=1,41,208,322,424,939 --cols=1,41,208,322,424,939

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...


In [20]:
!python stylegan2-ada/style_mixing.py --network={network_pkl} --outdir=out/mix_trunc0 --trunc=0.5 --rows=1,41,208,322,424,939 --cols=1,41,208,322,424,939

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...


In [30]:
from PIL import Image

image = Image.open('projector/im3.jpg')
new_image = image.resize((128, 128))
new_image.save('projector/im3.jpg')

In [31]:
!python stylegan2-ada/projector.py --network={network_pkl} --outdir=projector/3 --seed=0 --target=projector/im3.jpg

Loading networks from "/content/saves/00000-bonn-mirror-auto_tuned1-kimg50000/network-snapshot-004300.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 19.6403
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [00:57<00:00, 17.43it/s, dist=0.1409, loss=0.14]  
